# API Nakala

## Packages

In [ ]:
using CSV
using DataFrames
using HTTP
using JSON
using Dates

## Identifiants

In [ ]:
path = @__DIR__
credentials = CSV.read(joinpath(path,"credentials", "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = false

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Collections

### Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [ ]:
collectionName = ""

url = joinpath(apiurl, "collections")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :status => "private",
  :metas =>  [Dict(
    :value => collectionName,
    :propertyUri => "http://nakala.fr/terms#title",
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :lang => "fr"
  )]
)

postCollection = HTTP.request("POST", url, headers, JSON.json(body)) # envoi des données pour la création de la collection
collectionResponse = JSON.parse(String(HTTP.payload(postCollection))) # réponse du server
collectionId = collectionResponse["payload"]["id"] # récupération de l'id de la collection
print("Identifiant collection : ", collectionId)

### Ajouter des données à une collection

In [ ]:
collectionIdentifier = "10.34847/nkl.5c519k06"
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

postDatasToCollection = HTTP.request("POST", url, headers, JSON.json(body)) # ajoute les données listées ci-dessus à une collection
response = JSON.parse(String(HTTP.payload(postDatasToCollection))) # réponse du server

### Supprimer des données d'une collection

In [ ]:
collectionIdentifier = ""
datas = [
  "nakalaDataId_1",
  "nakalaDataId_2",
  "nakalaDataId_3"
]

url = joinpath(apiurl, "collections", collectionIdentifier, "datas")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = datas

removeDatasFromCollection = HTTP.request("DELETE", url, headers, JSON.json(body)) # supprime les données listées plus haut de la collection
response = JSON.parse(String(HTTP.payload(removeDatasFromCollection))) # réponse du server

## Utilisateurs

### Informations utilisateur

In [ ]:
url = joinpath(apiurl, "users", "me")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

userInfo = HTTP.request("GET", url, headers)
userInfoResponse = JSON.parse(String(HTTP.payload(userInfo))) # réponse du server
username = userInfoResponse["username"]
userGroupId = userInfoResponse["userGroupId"]
userInfoResponse

### Ressources utilisateur

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

#=
body = Dict(
  :page => 1,
  :limit => 100,
  :orders => [
    "creDate,desc"
  ],
  :types: => [
    "http://purl.org/coar/resource_type/c_c513"
  ],
  :status => [
    "published"
  ],
  :createdYears => [
    "2023"
  ],
  :collections => [
    "11280/9f85fbd6"
  ],
  :titleSearch => "",
  :titleSearchLang => "fr",
  :orderLang => "fr"
)
=#

body = Dict(
  :page => 1,
  :limit => 250
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

### Lister les données privées/publiques

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

status = [ "pending" ] #[ "pending", "published" ]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 300,
  :status => status
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

# récupération des données
datas = get(userDatasResponse, "data", "nothing")

pendingList = [get(data, "identifier", "") for data in datas]

### Publier les données privées

In [ ]:
# nécessite l'exécution de la cellule précédente.

for pendingData in pendingList
  headers = Dict(
    "X-API-KEY" => apiKey,
    "Content-Type" => "application/json"
  )
  
  url = joinpath(apiurl, "datas", pendingData, "status", "published")
  publishData = HTTP.put(url, headers)
  #publishDataResponse = JSON.parse(String(HTTP.payload(publishData))) # réponse du server 
end

### Lister les données

In [ ]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

#=
body = Dict(
  :page => 1,
  :limit => 100,
  :orders => [
    "creDate,desc"
  ],
  :types: => [
    "http://purl.org/coar/resource_type/c_c513"
  ],
  :status => [
    "published"
  ],
  :createdYears => [
    "2023"
  ],
  :collections => [
    "11280/9f85fbd6"
  ],
  :titleSearch => "",
  :titleSearchLang => "fr",
  :orderLang => "fr"
)
=#
# par défaut 100 premiers résultats
body = Dict(
  :page => 1,
  :limit => 500
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")

list = Vector()

for data in datas
  
  identifier = get(data, "identifier", "")
  metas = get(data, "metas", "")
  push!(m, metas)
  title = filter(x -> get(x, "propertyUri", "") == "http://nakala.fr/terms#title", metas)[1]

  item = Dict(
    get(title, "value", "noTitle") => identifier
  )
  push!(list, item)
end

list

### Chercher une donnée précise

In [ ]:
title = "Z1J432"

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => "fr",
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
datas[1]

### Chercher un fichier précis dans une donnée

In [ ]:
title = "Z1J432"
filenames = ["Z1J432_0004.JPG", "Z1J432_0005.JPG", "Z1J432_0006.JPG"]

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => "fr",
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
identifier = get(datas[1], "identifier", "")
filesList = get(datas[1], "files", "")

urls = Vector()
for file in filenames
  item = filter(x -> get(x, "name", "") == file, filesList)[1]
  fileIdentifier = get(item, "sha1", "")
  
  fileUrl = joinpath(apiurl, "data", identifier, fileIdentifier)
  push!(urls, fileUrl)
end

urls